# 1.numpy.nditer
`class numpy.nditer(op, flags=None, op_flags=None, op_dtypes=None, order='K', casting='safe', op_axes=None, itershape=None, buffersize=0)`

- **op**:ndarray 或 array_like 序列,需要迭代的数组
- **flags**:str序列，可选。用于控制迭代器行为的标志
  - `buffered` 在需要时启用缓冲
  - `c_index` 导致跟踪 C 顺序索引
  - `f_index` 导致跟踪 Fortran 顺序索引。
  - `multi_index` 导致跟踪一个多索引，或一个索引元组，每个迭代维度一个。
  - `external_loop`导致给定的值是具有多个值的一维数组，而不是零维数组。
  - `common_dtype`会将所有操作数转换为公共数据类型，并根据需要进行复制或缓冲。
  - 

- **op_flags**:str列表，可选。这是每个操作数的标志列表。至少，必须指定“readonly”，“readwrite”或“writeonly”中的一个。
  - `readwrite`表示将读取和写入操作数
  - `readonly`表示只读取操作数。
  - `writeonly`表示只会写入操作数。
  - `no_broadcast`阻止操作数被广播。

### 1.1默认迭代
以上实例不是使用标准 C 或者 Fortran 顺序，选择的顺序是和数组内存布局一致的，这样做的目的是为了提升访问的效率，默认是行顺序优先（row-major,或者说C-order）

In [1]:
import numpy as np

a = np.array([range(i,i+3) for i in [2,4,6]])
print(f"a =\n {a}\n")
  
print("迭代输出的元素")
for x in np.nditer(a):
    print(x,end=",")

a =
 [[2 3 4]
 [4 5 6]
 [6 7 8]]

迭代输出的元素
2,3,4,4,5,6,6,7,8,

这反映了默认情况下只需访问每个元素，而无需考虑其特定顺序。我们可以通过迭代上述数组的转置来看到这一点，并与以 C 顺序访问数组转置的 copy 方式做对比，如下实例

`a.T`为矩阵的转制

In [2]:
import numpy as np

a = np.random.random((3,4))
print('a=\n',a,'\n')
print("矩阵的转制")
print(a.T,'\n')

for idx in np.nditer(a.T.copy(order='C')):
    print(idx,end=',\n')

a=
 [[0.11476824 0.5105539  0.77588368 0.32751824]
 [0.48918067 0.62138278 0.56978691 0.6221257 ]
 [0.17921874 0.16896893 0.22558953 0.49966717]] 

矩阵的转制
[[0.11476824 0.48918067 0.17921874]
 [0.5105539  0.62138278 0.16896893]
 [0.77588368 0.56978691 0.22558953]
 [0.32751824 0.6221257  0.49966717]] 

0.11476824376656292,
0.48918067446960134,
0.17921874025296158,
0.5105539026096217,
0.6213827750861525,
0.16896893025811444,
0.7758836795920562,
0.5697869074373151,
0.22558952772457708,
0.3275182405236001,
0.6221256989049323,
0.4996671691612481,


### 1.2单维迭代

In [3]:
import numpy as np

x = np.arange(6).reshape(2,3)
it = np.nditer(x,flags=['f_index'])

while not it.finished:
    print(f"value = {it[0]},index = {it.index}")
    it.iternext()#it.iternext()表示进入下一次迭代，如果不加这句，输出结果就一直是0,并且不间断输出

value = 0,index = 0
value = 1,index = 2
value = 2,index = 4
value = 3,index = 1
value = 4,index = 3
value = 5,index = 5


### 1.3多维迭代

In [4]:
import numpy as np

x = np.arange(6).reshape(2,3)
print(f"x = \n{x}")
it = np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])

while not it.finished:
    print(f"value = {it[0]},index = {it.multi_index}")
    it.iternext()

x = 
[[0 1 2]
 [3 4 5]]
value = 0,index = (0, 0)
value = 1,index = (0, 1)
value = 2,index = (0, 2)
value = 3,index = (1, 0)
value = 4,index = (1, 1)
value = 5,index = (1, 2)


### 1.4列顺序迭代
`for x in np.nditer(a, order='F')`:Fortran order，即是列序优先；

In [5]:
import numpy as np
x = np.linspace(0,9,10).reshape(2,5)
print(f"x = {x}")

it = np.nditer(x,flags=['f_index'],order='F')
while not it.finished:
    print(f"value = {it[0]},index = {it.index}")
    it.iternext()
#可以看到输出结果是从列开始遍历

x = [[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
value = 0.0,index = 0
value = 5.0,index = 1
value = 1.0,index = 2
value = 6.0,index = 3
value = 2.0,index = 4
value = 7.0,index = 5
value = 3.0,index = 6
value = 8.0,index = 7
value = 4.0,index = 8
value = 9.0,index = 9


### 1.5行顺序迭代
`for x in np.nditer(a.T, order='C')`:C order，即是行序优先；

In [ ]:
import numpy as np

x = np.array([range(i,i+3) for i in [2,4,6]])
print(f"x = {x}")

it = np.nditer(x,flags=["c_index"],order="C")
while not it.finished:
    print(f"value = {it[0]},index = {it.index}")
    it.iternext()